In [12]:
import os

In [13]:
%pwd

'c:\\Users\\Portable\\OneDrive\\Desktop\\mlops\\End-to-End-Deep-Learning-Project-MLOps'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Portable\\OneDrive\\Desktop\\mlops\\End-to-End-Deep-Learning-Project-MLOps'

In [21]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/BouazizOmar/End-to-End-Deep-Learning-Project-MLOps.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="bouazizomar"
os.environ["MLFLOW_TRACKING_PASSWORD"]=""

In [22]:
import tensorflow as tf 

In [23]:
model = tf.keras.models.load_model(r"artifacts\training\model.h5")

In [31]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass
class EvaluationConfig:
    path_of_model: Path 
    training_data: Path 
    all_params: dict 
    mlflow_uri: str 
    params_image_size: list 
    params_batch_size: int 

In [32]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
        path_of_model=r"artifacts\training\model.h5",
        training_data=r"artifacts\data_ingestion\train",
        mlflow_uri="https://dagshub.com/BouazizOmar/End-to-End-Deep-Learning-Project-MLOps.mlflow",
        all_params=self.params,
        params_image_size=self.params.IMAGE_SIZE,
        params_batch_size=self.params.BATCH_SIZE)
        return eval_config

In [35]:
import tensorflow as tf
from pathlib import Path 
import mlflow 
import mlflow.keras 
from urllib.parse import urlparse

class Evaluation:
    def __init__(self, config:EvaluationConfig):
        self.config = config 
        
    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )
        
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()
    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            if tracking_url_type_store != "file":
                # track in the remote server
                print("tracking url: ",tracking_url_type_store)
                print("***remote***\n")
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                # track in the local
                print("***local***\n")
                mlflow.keras.log_model(self.model, "model")
                    
        

In [36]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-09-01 20:01:23,631: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-01 20:01:23,634: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-01 20:01:23,636: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 12s 2s/step - loss: 1.4060 - accuracy: 0.6078
[2024-09-01 20:01:35,885: INFO: common: json file saved at: scores.json]


2024/09/01 20:01:36 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


tracking url:  https
***remote***

[2024-09-01 20:01:37,756: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\Portable\AppData\Local\Temp\tmpp440j3uq\model\data\model\assets
[2024-09-01 20:01:39,518: INFO: builder_impl: Assets written to: C:\Users\Portable\AppData\Local\Temp\tmpp440j3uq\model\data\model\assets]


KeyboardInterrupt: 